# Exercise

In this exercise, we will build a small graph convolutional neural network using PyTorch. 

## Graph Convolutional neural network 

A graph convolutional neural network (GCN) is a neural network designed to work with network data. It extends traditional convolutional neural networks (CNNs) that operate on grid-like data such as images to general non-grid-like graphs. 

We will implement a popular model proposed by Kipf and Welling in ICLR 2017: 

- [Kipf, Thomas and Max Welling. "Semi-Supervised Classification with Graph Convolutional Networks." ArXiv abs/1609.02907 (2016)](https://openreview.net/pdf?id=SJU4ayYgl)


## Data

We will use the airport network as the example data.  

In [ ]:
import pandas as pd
import numpy as np
from scipy import sparse

node_table = pd.read_csv(
    "https://raw.githubusercontent.com/skojaku/adv-net-sci-course/main/data/airport_network_v2/node_table.csv"
)
edge_table = pd.read_csv(
    "https://raw.githubusercontent.com/skojaku/adv-net-sci-course/main/data/airport_network_v2/edge_table.csv",
    dtype={"src": np.int32, "trg": np.int32},
)
src, trg = tuple(edge_table[["src", "trg"]].values.T)

rows, cols = src, trg
nrows, ncols = node_table.shape[0], node_table.shape[0]
A = sparse.csr_matrix(
    (np.ones_like(rows), (rows, cols)),
    shape=(nrows, ncols),
).asfptype()

# Symmterize and binarize
A = A + A.T
A.data = A.data * 0 + 1

node_labels = node_table["region"].values
node_label_ids = np.unique(node_table["region"].values, return_inverse=True)[1]
num_classes = len(np.unique(node_label_ids))

## Implementing GCN 

The Graph Convolutional Network (GCN) carries out the convolution of node features in the following steps:

1. The initial vector of node features, which has a dimension of $K$, is transformed into a new vector of dimension $K_h$ using a fully connected linear layer that can be learned. 
2. GCN performs a convolutional operation by convolving the transformed feature vector generated in step 1 with the feature vectors of its neighboring nodes. This generates a new feature vector for each node. 
3. The convoluted vectors are then passed through a linear or non-linear transformation.
4. The steps 1--3 are repeated multiple times
5. The final node features are fed into a linear layer, and then the last layer for a downstream application (e.g., soft-max for a classification task) 

Let's implement it one by one by using PyTorch.

### Step 0: Preparation
To demonstrate the effectiveness of convolution, let's use a random node feature generated by a Gaussian distribution. This feature is 100 dimensional and does not contain any information about the network. We will see that, even with this random node feature, GCN performs well in a downstream task due to its ability to leverage the network structure. 

In [ ]:
import torch

dim = 100
X = torch.randn(A.shape[0], dim)  # Random features for each node



### Step 1: Linear transformation

Let us consider that each node $i$ has a $K$-dimensional feature vector $x_i$ of continuous variables. Similarly, let us denote by $X$ the feature matrix whose $i$th row corresponds to the feature vector of node i. 

$$
X := 
\begin{bmatrix}
x_i \\
x_2 \\
\vdots \\
x_i \\
\vdots \\
x_n \\
\end{bmatrix}
= 
\begin{bmatrix}
x_{11} & x_{12} & \cdots & x_{1K}\\
x_{21} & x_{22} & \cdots & x_{2K}\\
\vdots & \cdots & \ddots & \vdots\\
x_{i1} & x_{i2} & \cdots & x_{iK}\\
\vdots & \cdots & \ddots & \vdots\\
x_{N1} & x_{N2} & \cdots & x_{NK}\\
\end{bmatrix}
$$

where $N$ is the number of nodes. 

GCN first applies a linear transformation to $X$, namely 
$$
z^{(1)}_i = x_i W,\quad\text{or}\quad Z^{(1)} = X W
$$
where $W$ is a $K\times K'$ learnable matrix, and $Z^{(1)}$ is a new feature vector of nodes. The "learnable" means that matrix $W$ will be adjusted to improve the performance of a downstream task. To implement this step, we will use [PyTorch implementation of linear layer](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html). 

In [ ]:
import torch

# This is the dimension of the linear layer W
in_features = 100  # This is the dimension of the input features
out_features = 50  # This is the dimension of the output features

# Create a linear layer
# The bias is set to False to be faithful to the original formulation. But you can set it to True if you want.
linear_layer = torch.nn.Linear(
    in_features=in_features, out_features=out_features, bias=False
)

You can apply the linear layer by calling it a function.

In [ ]:
Z1 = linear_layer(X)
Z1

### Step 2: Aggregation of node features

The next step is the convolution. GCN generates a new feature vector $z_{i}^{(2)}$ for a node $i$ by aggregating the node features of the node itself (i.e., $z_i^{(1)}$) and those of its neighboring nodes. While one can use any aggregation type, such as sum, averaging, or max/min pooling, this specific GCN adopted a weighted average based on the normalized adjacency matrix with loops. Specifically, 

$$
z^{(2)}_i = \sum_{j} \tilde{A}^{\text{norm}}_{ij} z^{(1)} _j, 
$$
or equivalently, in matrix form, 
$$
Z^{(2)} = \tilde{A}^{\text{norm}} Z^{(1)}, 
$$

Here, $\tilde{A}^{\text{norm}}$ represents the normalized adjacency matrix and is defined as 

$$ \tilde{A}^{\text{norm}} = \tilde{D}^{-1/2} \tilde{A} \tilde{D}^{-1/2}. $$

Matrix $\tilde{A} = A + I$ is the adjacency matrix with self-loops added (represented by $I$), and $\tilde{D} = \text{diag}\tilde (d_1,\ldots, \tilde d_N)$ is the diagonal matrix that contains the degrees of the nodes in the adjacency matrix with self-loops (i.e., $\tilde A$). 

(Self-loops are added to nodes for a specific reason. When a self-loop is added to a node, the node itself becomes one of its neighbors. As a result, the convolution is defined by combining the features of the node's neighbors, and we can safely disregard the inclusion of the node's features.)

Now, let's implement the convolusion. We will construct the normalized adjacency matrix by using scipy and numpy. But since the node features are PyTorch Tensor, we will transform $\tilde A$ in scipy.sparse.csr_matrix format into the corresponding format in PyTorch by using the following function:

In [ ]:
# Use this function to convert scipy sparse matrix to torch sparse matrix
def to_torch_sparse(A):
    """Convert scipy sparse matrix to torch sparse matrix"""
    Atorch = torch.sparse_csr_tensor(A.indptr, A.indices, A.data, dtype=torch.float32)
    return Atorch

Now, let's construct the normalized adjacency matrix with self-loops ($\tilde A$).  

In [ ]:
# Your code -----
# Construct the normalized adjacency matrix with self-loops
Atilde = ...  # Adjacency matrix with self-loops
Dtilde_inv_sqrt = ...  # Inverse square root of Dtilde
Atilde_norm = ...  # Normalized adjacency matrix
Atilde_norm = to_torch_sparse(Atilde_norm)

Let's perform the convolution.  

In [ ]:
Z2 = Atilde_norm @ Z1

By this convolution, the new feature $Z^{(2)}$ reflects not only the original node features but also the network structures. 

### Step 3: Non-linear activation 

GCN applies a non-linear activation to the aggregated feature vector. Here, we use the [ReLu](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html) as the activation function by following the original implementation. 

In [ ]:
activation_layer = torch.nn.ReLU()

As for the linear layer, the `activation_layer` is a function that applies ReLu to its argument.  

In [ ]:
activation_layer(Z2)

### Step 4: Stacking multiple GCN layers

Let's combine the components into a single GCN layer using the [torch.nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html) class in PyTorch. 

In [ ]:
def GCN_forward(Atilde_norm, X, linear_layer, activation_layer):
    """Forward pass"""
    Z1 = linear_layer(X)
    Z2 = Atilde_norm @ Z1
    Z3 = activation_layer(Z2)
    return Z3


# Test
GCN_forward(Atilde_norm, X, linear_layer, activation_layer)

We can stack multiple GCN layers to perform the convolution multiple times. By stacking GCN layers, a focal node is able to propagate its features to its immediate neighbors through the first convolution. Through the second convolution, the focal node's features are further propagated to the neighbors of its neighbors. This enables the aggregated node feature to reflect information from a wider range of nodes in the network.

Let's perform the GCN operation multiple times to see the benefit of stacking GCN layers. We will define the GCN operation as follows:

In [ ]:
linear_layer = torch.nn.Linear(100, 100)
activation_layer = torch.nn.ReLU()
GCN_forward(Atilde_norm, X, linear_layer, activation_layer)

Let's perform the graph convolution multiple times. At each application of the graph convolution, we will project the 100-dimensional feature vector in 2D to see the structure encoded in the feature vectors. Note that GCN is not trained at all! 

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import matplotlib.pyplot as plt
import seaborn as sns

n_conv = 5
Z = X.clone()
xylist = []
for i in range(n_conv):
    Z = GCN_forward(Atilde_norm, Z, linear_layer, activation_layer)
    xy = LinearDiscriminantAnalysis(n_components=2).fit_transform(
        Z.detach().numpy(), node_label_ids
    )
    xylist.append(xy)

sns.set_style("white")
sns.set(font_scale=1.2)
sns.set_style("ticks")
fig, axes = plt.subplots(figsize=(4 * len(xylist), 4), ncols=len(xylist))

for i, xy in enumerate(xylist):
    ax = axes[i]
    sns.scatterplot(x=xy[:, 0], y=xy[:, 1], hue=node_table["region"], ax=ax)
    ax.set_title(f"{i}th convolution")
    ax.set_xlabel("PC1")
    ax.set_ylabel("PC2")

    ax.legend().remove()

ax.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.0, frameon=False)
sns.despine()

By repeating the convolution process, initially random feature vectors (0th iteration) gradually form clusters by regions because each convolution step reflects the network structure into the aggregation.  

### Step 5: Transforming the aggregated features to an output

The last step is to transform the aggregated node feature vectors into an output vector (or variable). It is common that the aggregated feature is once sent to a linear layer. Then, the linear-transformed aggregated node features are sent to the final layer for a downstream application. 
We will apply the GCN for a classification task using the [Soft-max function](https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html).  

In [ ]:
second_last_layer = torch.nn.Linear(
    100, num_classes
)  # This is the dimension of the output features
out_layer = torch.nn.Softmax()

In [ ]:
out_layer(second_last_layer(Z))

## Assemble GCN

Let's assemble all GCN components for a classification task. In this task, we will predict the region of individual airports. 
Following the PyTorch style, we will create the GCN using [torch.nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html).  

In [ ]:
import torch
from torch.nn import Linear, LeakyReLU, Softmax
import torch.nn.functional as F
import numpy as np
from scipy.sparse import coo_matrix


class GraphConv(torch.nn.Module):
    def __init__(self, in_channels, out_channels, A):
        """Graph Convolution Layer

        Parameters
        ----------
        in_channels : int
         Input dimension
        out_channels : int
         Output dimension
        A : scipy.csr_matrix (n_nodes, n_nodes)
         Adjacency matrix
        """
        super(GraphConv, self).__init__()
        # Your code ----
        self.conv_mat = ...
        self.linear = ...
        self.act = ...
        # --------------

    def forward(self, x):
        """Forward pass

        Parameters
        ----------
        x : torch.Tensor (n_nodes, in_channels)
         Input node features

        Returns
        -------
        torch.Tensor (n_nodes, out_channels)
         Output node features
        """

        # Your code ----
        z = ...
        # --------------

        return z


# Test
def test_GCN():
    GraphConv(100, 50, A)


test_GCN()

Next, by using the GCNLayer defined above, assemble a graph neural network with two GCNLayers. The first GCN layer transforms feature vectors from `in_channel` dimensions to `hidden_channel` dimensions, and the second GCN layer transforms the `hidden_channel` dimensional vector to `out_channel` dimensional vectors. Finally, a linear layer is applied to transform the `out_channel` dimensional vector to `out_channel` dimensional vector, which is then sent to the Softmax layer to produce the final output for classification.  

In [ ]:
import torch
from torch.nn import Linear, LeakyReLU, Softmax
import torch.nn.functional as F
import numpy as np


class GCN(torch.nn.Module):
    def __init__(self, in_channel, hidden_channel, out_channel, A):
        """Graph Convolution Network

        Parameters
        ----------
        in_channel : int
         Input dimension
        hidden_channel : int
         Hidden dimension
        out_channel : int
         Output dimension
        A : scipy.csr_matrix (n_nodes, n_nodes)
         Adjacency matrix
        """
        super(GCN, self).__init__()
        self.in_channel = in_channel
        self.hidden_channel = hidden_channel
        self.out_channel = out_channel

        # Your code ----
        # Hint
        # self.conv1 = ...
        # self.conv2 = ...
        # self.fully_connected = ...
        # self.softmax = Softmax(dim = 1)
        # --------------

    def forward(self, x):
        """Forward pass

        Parameters
        ----------
        x : torch.Tensor (n_nodes, in_channels)
         Input node features

        Returns
        -------
        torch.Tensor (n_nodes, out_channels)
         Output node features
        """

        # Your code ----
        # --------------

        return z

## Train GCN  

We will first split the nodes into train and test sets. 

In [ ]:
# Split the node table into the train and test set.
df = node_table.sample(frac=1, random_state=0)
train_node_table = df.iloc[: int(len(df) * 0.8)]
test_node_table = df.iloc[int(len(df) * 0.2) :]

Let's train GCN. We first construct the GCN model.  

In [ ]:
n_labels = len(np.unique(node_labels))
hidden_channel = 100
in_channel = X.shape[1]
out_channel = n_labels
model = GCN(
    in_channel=in_channel, hidden_channel=hidden_channel, out_channel=out_channel, A=A
)

Then, train the GCN by 

In [ ]:
from torch import optim
from tqdm.auto import tqdm


# Define training loop
def train(model, optimizer, criterion, x_train, y_train, A, train_mask):
    """Train the model

    Parameters
    ----------
    model : torch.nn.Module
     Model
    optimizer : torch.optim.Optimizer
     Optimizer
    criterion : torch.nn.modules.loss._Loss
     Loss function
    x_train : torch.Tensor (n_nodes, in_channels)
     Input node features
    y_train : torch.Tensor (n_nodes)
     True labels
    A : scipy.sparse.csr_matrix (n_nodes, n_nodes)
     Adjacency matrix
    train_mask : numpy.ndarray
     Mask for training nodes

    Returns
    -------
    loss.item() : float
     Loss value
    """

    # Reset gradient
    optimizer.zero_grad()

    # Forward pass
    output = model(x_train)

    # Only compute loss for nodes in the training set
    loss = criterion(output[train_mask, :], y_train[train_mask])

    # Backward pass
    loss.backward()

    # Update parameters
    optimizer.step()

    # Return loss
    return loss.item()


# Define loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Find the indices of the train and test nodes
train_mask = np.array(train_node_table["node_id"])
test_mask = np.array(test_node_table["node_id"])

# Convert numpy arrays to torch tensors
Y = torch.LongTensor(node_label_ids)

# Number of epochs to train
n_epochs = 1000
pbar = tqdm(range(n_epochs))

# Train the model
model.train()
for epoch in pbar:
    loss = train(model, optimizer, criterion, X, Y, A, train_mask)
    pbar.set_description(f"Epoch {epoch+1}, Loss: {loss:.4f}")

Let's evaluate the performance of the model by using accuracy. 

In [ ]:
def eval_prediction_accuracy(y, yred):
    """Calculate prediction accuracy.

    Parameters
    ----------
    y : numpy.ndarray
     True labels.
    ypred : numpy.ndarray
     Predicted labels.

    Returns
    -------
    acc : float
     Prediction accuracy.
    """
    return float(np.sum(y == yred)) / float(len(y))

In [ ]:
# Evaluate the model
model.eval()

# Your code ----
# Hint
# output = ...
# ypred = ... # output gives a probability distribution over classes. Pick the one with the highest probability.
# --------------

acc = eval_prediction_accuracy(Y[test_mask].numpy(), ypred[test_mask])
print(f"Test accuracy: {acc:.4f}")

## Different node features

While we have used a random node feature that has not contributed to any application, the GCN performs well for the classification since it leverages the network structure. This time, we give the GCN another useful node feature for classification, namely the latitude, longitude, and Time zone.  

In [ ]:
X = node_table[["Latitude", "Longitude", "Timezone"]].values
X = torch.FloatTensor(X)

Let's train the GCN and check out the performance. 

# Different configuration of the GCN. 

GCN has many user-defined hyperparameters, such as the dimensionality of the hidden layers, type of non-linear activations, and number of GCN layers.
Change these parameters and see how sensitive these hyperparameters are to the performance. 